In [2]:
import os
import json

import pandas as pd
import numpy as np

from tqdm import tqdm
from gensim.models import Word2Vec
from gensim.models.keyedvectors import WordEmbeddingsKeyedVectors

In [3]:
import io
import os
import json
import distutils.dir_util
from collections import Counter

import numpy as np


def write_json(data, fname):
    def _conv(o):
        if isinstance(o, (np.int64, np.int32)):
            return int(o)
        raise TypeError

    parent = os.path.dirname(fname)
    distutils.dir_util.mkpath("./arena_data/" + parent)
    with io.open("./arena_data/" + fname, "w", encoding="utf-8") as f:
        json_str = json.dumps(data, ensure_ascii=False, default=_conv)
        f.write(json_str)


def load_json(fname):
    with open(fname, encoding="utf-8") as f:
        json_obj = json.load(f)

    return json_obj


def debug_json(r):
    print(json.dumps(r, ensure_ascii=False, indent=4))


def remove_seen(seen, l):
    seen = set(seen)
    return [x for x in l if not (x in seen)]


def most_popular(playlists, col, topk_count):
    c = Counter()

    for doc in playlists:
        c.update(doc[col])

    topk = c.most_common(topk_count)
    return c, [k for k, v in topk]

In [8]:
class PlaylistEmbedding:
    def __init__(self, FILE_PATH):
        self.FILE_PATH = FILE_PATH
        self.min_count = 3
        self.size = 100
        self.window = 210
        self.sg = 5
        self.p2v_model = WordEmbeddingsKeyedVectors(self.size)
        with open(os.path.join(FILE_PATH, 'train.json'), encoding="utf-8") as f:
            self.train = json.load(f)
        with open(os.path.join(FILE_PATH, 'val.json'), encoding="utf-8") as f:
            self.val = json.load(f)
        with open(os.path.join("./../collaborative_filtering", 'results.json'), encoding="utf-8") as f:
            self.most_results = json.load(f)
            
    def get_dic(self, train, val):
        song_dic = {}
        tag_dic = {}
        data = train + val
        for q in tqdm(data):
            song_dic[str(q['id'])] = q['songs']
            tag_dic[str(q['id'])] = q['tags']
        self.song_dic = song_dic
        self.tag_dic = tag_dic
        total = list(map(lambda x: list(map(str, x['songs'])) + list(x['tags']), data))
        total = [x for x in total if len(x)>1]
        self.total = total
        
    def get_w2v(self, total, min_count, size, window, sg):
        w2v_model = Word2Vec(total, min_count = min_count, size = size, window = window, sg = sg)
        self.w2v_model = w2v_model
            
    def update_p2v(self, train, val, w2v_model):
        ID = []   
        vec = []
        for q in tqdm(train + val):
            tmp_vec = 0
            if len(q['songs'])>=1:
                for song in q['songs'] + q['tags']:
                    try: 
                        tmp_vec += w2v_model.wv.get_vector(str(song))
                    except KeyError:
                        pass
            if type(tmp_vec)!=int:
                ID.append(str(q['id']))    
                vec.append(tmp_vec)
        self.p2v_model.add(ID, vec)
    
    def get_result(self, p2v_model, song_dic, tag_dic, most_results, val):
        answers = []
        for n, q in tqdm(enumerate(val), total = len(val)):
            try:
                most_id = [x[0] for x in p2v_model.most_similar(str(q['id']), topn=200)]
                get_song = []
                get_tag = []
                for ID in most_id:
                    get_song += song_dic[ID]
                    get_tag += tag_dic[ID]
                get_song = list(pd.value_counts(get_song)[:200].index)
                get_tag = list(pd.value_counts(get_tag)[:20].index)
                answers.append({
                    "id": q["id"],
#                     "songs": remove_seen(q["songs"], get_song)[:100],
                    "songs": most_results[n]['songs'][:100],
                    "tags": remove_seen(q["tags"], get_tag)[:10],
                })
            except:
                answers.append({
                  "id": most_results[n]["id"],
                  "songs": most_results[n]['songs'],
                  "tags": most_results[n]["tags"],
                }) 
        # check and update answer
        for n, q in enumerate(answers):
            if len(q['songs'])!=100:
                answers[n]['songs'] += remove_seen(q['songs'], self.most_results[n]['songs'])[:100-len(q['songs'])]
            if len(q['tags'])!=10:
                answers[n]['tags'] += remove_seen(q['tags'], self.most_results[n]['tags'])[:10-len(q['tags'])]  
        self.answers = answers
    
    def run(self):
        self.get_dic(self.train, self.val)
        self.get_w2v(self.total, self.min_count, self.size, self.window, self.sg)
        self.update_p2v(self.train, self.val, self.w2v_model)
        self.get_result(self.p2v_model, self.song_dic, self.tag_dic, self.most_results, self.val)
        write_json(self.answers, 'results.json')

In [9]:
FILE_PATH = './../data/'
U_space = PlaylistEmbedding(FILE_PATH)
# U_space.run()
U_space.get_dic(U_space.train, U_space.val)

100%|██████████| 138086/138086 [00:00<00:00, 141784.29it/s]


In [ ]:
# U_space.get_w2v(U_space.total, U_space.min_count, U_space.size, U_space.window, U_space.sg)

# # save my model
# U_space.w2v_model.save('w2v_0614.model')

In [10]:
U_space.w2v_model = Word2Vec.load('w2v_0614.model')

In [11]:
U_space.update_p2v(U_space.train, U_space.val, U_space.w2v_model)


 19%|█▊        | 25717/138086 [00:14<01:14, 1515.90it/s]


 40%|███▉      | 54995/138086 [00:29<00:50, 1657.76it/s]


 62%|██████▏   | 85720/138086 [00:43<00:29, 1772.55it/s]


 80%|███████▉  | 109861/138086 [00:59<00:20, 1353.44it/s]


100%|██████████| 138086/138086 [01:07<00:00, 2033.19it/s]


In [12]:
U_space.p2v_model

In [13]:
U_space.get_result(U_space.p2v_model, U_space.song_dic, U_space.tag_dic, U_space.most_results, U_space.val)


  3%|▎         | 741/23015 [00:18<10:48, 34.34it/s]


  8%|▊         | 1858/23015 [00:36<02:56, 119.62it/s]


 15%|█▌        | 3476/23015 [00:54<03:15, 100.15it/s]


 22%|██▏       | 4997/23015 [01:10<03:37, 82.69it/s]


 28%|██▊       | 6470/23015 [01:28<06:16, 43.93it/s]


...

 80%|███████▉  | 18402/23015 [03:39<00:51, 89.41it/s]


 86%|████████▋ | 19867/23015 [03:55<00:36, 86.67it/s]


 93%|█████████▎| 21384/23015 [04:11<00:19, 84.89it/s]


 99%|█████████▉| 22820/23015 [04:28<00:02, 75.00it/s]


100%|██████████| 23015/23015 [04:30<00:00, 85.12it/s]


In [16]:
write_json(U_space.answers, 'results.json')

In [14]:
U_space.most_results[0]

{'id': 118598,
 'songs': [207912,
  623047,
  422438,
  703323,
  638488,
  322215,
  140837,
  690767,
  157900,
  39436,
  516376,
  11657,
  439161,
  78983,
  569715,
  394031,
  413459,
  213435,
  328908,
  472374,
  217622,
  262430,
  58773,
  557956,
  66475,
  207558,
  209622,
  494552,
  236393,
  607459,
  551157,
  367826,
  411438,
  292859,
  654757,
  448547,
  385871,
  387859,
  76888,
  187531,
  358186,
  326204,
  205757,
  438857,
  695032,
  542127,
  693988,
  571016,
  447762,
  140867,
  625875,
  459256,
  654428,
  620311,
  272379,
  307938,
  616144,
  32120,
  412246,
  545816,
  646988,
  572238,
  671973,
  464051,
  333595,
  570479,
  365613,
  421833,
  249378,
  55791,
  455806,
  1133,
  219415,
  284913,
  376435,
  585728,
  255937,
  112399,
  680970,
  61771,
  613740,
  400781,
  554751,
  278184,
  493762,
  224921,
  640239,
  146989,
  467269,
  5049,
  226991,
  468494,
  375776,
  424080,
  21512,
  70624,
  673026,
  679436,
  396028,
 

In [15]:
U_space.answers[0]

{'id': 118598,
 'songs': [207912,
  623047,
  422438,
  703323,
  638488,
  322215,
  140837,
  690767,
  157900,
  39436,
  516376,
  11657,
  439161,
  78983,
  569715,
  394031,
  413459,
  213435,
  328908,
  472374,
  217622,
  262430,
  58773,
  557956,
  66475,
  207558,
  209622,
  494552,
  236393,
  607459,
  551157,
  367826,
  411438,
  292859,
  654757,
  448547,
  385871,
  387859,
  76888,
  187531,
  358186,
  326204,
  205757,
  438857,
  695032,
  542127,
  693988,
  571016,
  447762,
  140867,
  625875,
  459256,
  654428,
  620311,
  272379,
  307938,
  616144,
  32120,
  412246,
  545816,
  646988,
  572238,
  671973,
  464051,
  333595,
  570479,
  365613,
  421833,
  249378,
  55791,
  455806,
  1133,
  219415,
  284913,
  376435,
  585728,
  255937,
  112399,
  680970,
  61771,
  613740,
  400781,
  554751,
  278184,
  493762,
  224921,
  640239,
  146989,
  467269,
  5049,
  226991,
  468494,
  375776,
  424080,
  21512,
  70624,
  673026,
  679436,
  396028,
 